# Загрузка

In [3]:
import math
import random
#import Params
import asyncio

from datetime import datetime 

In [ ]:
def createFile(F) :
    #from datetime import datetime 
    now = datetime.now()
    current_time = now.strftime("%m%d_h%Hm%M")

    f = open(F+current_time+'.txt','w')
    return f

## Соединение с биржей

In [4]:


import ssl
import time
import json
import urllib
import hmac, hashlib
import requests
 
from urllib.parse import urlparse, urlencode
from urllib.request import Request, urlopen
 
class Binance():
 
    methods = {
            # public methods
            'ping':             {'url':'api/v1/ping', 'method': 'GET', 'private': False},
            'time':             {'url':'api/v1/time', 'method': 'GET', 'private': False},
            'exchangeInfo':     {'url':'api/v1/exchangeInfo', 'method': 'GET', 'private': False},
            'depth':            {'url': 'api/v1/depth', 'method': 'GET', 'private': False},
            'trades':           {'url': 'api/v1/trades', 'method': 'GET', 'private': False},
            'historicalTrades': {'url': 'api/v1/historicalTrades', 'method': 'GET', 'private': False},
            'aggTrades':        {'url': 'api/v1/aggTrades', 'method': 'GET', 'private': False},
            'klines':           {'url': 'api/v1/klines', 'method': 'GET', 'private': False},
            'ticker24hr':       {'url': 'api/v1/ticker/24hr', 'method': 'GET', 'private': False},
            'tickerPrice':      {'url': 'api/v3/ticker/price', 'method': 'GET', 'private': False},
            'tickerBookTicker': {'url': 'api/v3/ticker/bookTicker', 'method': 'GET', 'private': False},
            # private methods
            'createOrder':      {'url': 'api/v3/order', 'method': 'POST', 'private': True},
            'testOrder':        {'url': 'api/v3/order/test', 'method': 'POST', 'private': True},
            'orderInfo':        {'url': 'api/v3/order', 'method': 'GET', 'private': True},
            'cancelOrder':      {'url': 'api/v3/order', 'method': 'DELETE', 'private': True},
            'openOrders':       {'url': 'api/v3/openOrders', 'method': 'GET', 'private': True},
            'allOrders':        {'url': 'api/v3/allOrders', 'method': 'GET', 'private': True},
            'account':          {'url': 'api/v3/account', 'method': 'GET', 'private': True},
            'myTrades':         {'url': 'api/v3/myTrades', 'method': 'GET', 'private': True},
            # wapi
            'depositAddress':   {'url': 'wapi/v3/depositAddress.html', 'method':'GET', 'private':True},
            'withdraw':   {'url': 'wapi/v3/withdraw.html', 'method':'POST', 'private':True},
            'depositHistory': {'url': 'wapi/v3/depositHistory.html', 'method':'GET', 'private':True},
            'withdrawHistory': {'url': 'wapi/v3/withdrawHistory.html', 'method':'GET', 'private':True},
            'assetDetail': {'url': 'wapi/v3/assetDetail.html', 'method':'GET', 'private':True},
            'tradeFee': {'url': 'wapi/v3/tradeFee.html', 'method':'GET', 'private':True},
            'accountStatus': {'url': 'wapi/v3/accountStatus.html', 'method':'GET', 'private':True},
            'systemStatus': {'url': 'wapi/v3/systemStatus.html', 'method':'GET', 'private':True},
            'assetDust': {'url': 'sapi/v1/asset/dust', 'method':'POST', 'private':True},
            'dustLog': {'url': 'wapi/v3/userAssetDribbletLog.html', 'method':'GET', 'private':True},
            'assetAssetDividend': {'url': 'sapi/v1/asset/assetDividend', 'method':'GET', 'private':True},
            #sapi
            'marginTransfer': {'url': 'sapi/v1/margin/transfer', 'method': 'POST', 'private':True},
            'marginLoan': {'url': 'sapi/v1/margin/loan', 'method': 'POST', 'private': True},
            'marginLoanGet': {'url': 'sapi/v1/margin/loan', 'method': 'GET', 'private': True},
            'marginRepay': {'url': 'sapi/v1/margin/repay', 'method': 'POST', 'private': True},
            'marginRepayGet': {'url': 'sapi/v1/margin/repay', 'method': 'GET', 'private': True},
            'marginCreateOrder': {'url': 'sapi/v1/margin/order', 'method': 'POST', 'private':True},
            'marginCancelOrder': {'url': 'sapi/v1/margin/order', 'method': 'DELETE', 'private':True},
            'marginOrderInfo': {'url': 'sapi/v1/margin/order', 'method': 'GET', 'private':True},
            'marginAccount': {'url': 'sapi/v1/margin/account', 'method': 'POST', 'private':True},
            'marginOpenOrders': {'url': 'sapi/v1/margin/openOrders', 'method': 'GET', 'private':True},
            'marginAllOrders': {'url': 'sapi/v1/margin/allOrders', 'method': 'GET', 'private':True},
            'marginAsset': {'url': 'sapi/v1/margin/asset', 'method': 'POST', 'private':True},
            'marginPair': {'url': 'sapi/v1/margin/pair', 'method': 'POST', 'private':True},
            'marginPriceIndex': {'url': 'sapi/v1/margin/priceIndex', 'method': 'POST', 'private':True},
            'marginMyTrades': {'url': 'sapi/v1/margin/myTrades', 'method': 'GET', 'private':True},
            'marginMaxBorrowable': {'url': 'sapi/v1/margin/maxBorrowable', 'method': 'GET', 'private':True},
            'marginmaxTransferable': {'url': 'sapi/v1/margin/maxTransferable', 'method': 'GET', 'private':True},
            #futures
            'futuresExchangeInfo': {'url': 'fapi/v1/exchangeInfo', 'method': 'GET', 'private': False, 'futures': True},
            'futuresKlines': {'url': 'fapi/v1/klines', 'method': 'GET', 'private': False, 'futures': True},
            'futuresCreateOrder':      {'url': 'fapi/v1/order', 'method': 'POST', 'private': True, 'futures': True},
            'futuresAccount':      {'url': 'fapi/v1/account', 'method': 'POST', 'private': True, 'futures': True},
            'futuresBalance':      {'url': 'fapi/v1/balance', 'method': 'GET', 'private': True, 'futures': True},
            'futuresSymbolPriceTicker': {'url': 'fapi/v1/ticker/price', 'method': 'GET', 'private': True, 'futures': True},
            'futuresOrderInfo': {'url': 'fapi/v1/order', 'method': 'GET', 'private': True, 'futures': True},
            'futuresCancelOrder':      {'url': 'fapi/v1/order', 'method': 'DELETE', 'private': True, 'futures': True},
   }
    
    def __init__(self, API_KEY, API_SECRET, bname):
        self.API_KEY = API_KEY
        self.API_SECRET = bytearray(API_SECRET, encoding='utf-8')
        self.shift_seconds = 0
        self.bname = bname
 
    def __getattr__(self, name):
        def wrapper(*args, **kwargs):
            kwargs.update(command=name)
            return self.call_api(**kwargs)
        return wrapper
 
    def set_shift_seconds(self, seconds):
        self.shift_seconds = seconds
        
    def call_api(self, **kwargs):
 
        command = kwargs.pop('command')
 
        base_url ='https://api.binance.com/' 
        if self.methods[command].get('futures'):
            base_url = 'https://fapi.binance.com/' 
        api_url = base_url  + self.methods[command]['url']
 
        payload = kwargs
        headers = {}
        
        payload_str = urllib.parse.urlencode(payload)
        if self.methods[command]['private']:
            payload.update({'timestamp': int(time.time() + self.shift_seconds - 1) * 1000})
            payload_str = urllib.parse.urlencode(payload).encode('utf-8')
            sign = hmac.new(
                key=self.API_SECRET,
                msg=payload_str,
                digestmod=hashlib.sha256
            ).hexdigest()
 
            payload_str = payload_str.decode("utf-8") + "&signature="+str(sign) 
            headers = {"X-MBX-APIKEY": self.API_KEY, "Content-Type":"application/x-www-form-urlencoded"}
 
        if self.methods[command]['method'] == 'GET' or self.methods[command]['url'].startswith('sapi'):
            api_url += '?' + payload_str
 
        response = requests.request(method=self.methods[command]['method'], url=api_url, data="" if self.methods[command]['method'] == 'GET' else payload_str, headers=headers)
            
        if 'code' in response.text:
            raise Exception(response.text)
 
        return response.json()

## Ключи

In [3]:
# кошелек, откуда переливаем
bot1 = Binance( API_KEY = b1_API_KEY, API_SECRET = b1_API_SECRET, bname = 'b1' )
    
# кошелек, куда переливаем
bot2 = Binance( API_KEY = b2_API_KEY, API_SECRET = b2_API_SECRET, bname = 'b2' )    

NameError: name 'b1_API_KEY' is not defined

## ФИЛЬТРЫ : Поиск списка пар с хорошим спрэдом

In [ ]:
# Поcтроение словаря с настройками всех торговых пар.

sett = {}

eI = bot2.exchangeInfo()['symbols'] # Настройки и лимиты биржи

for j in range( len(eI) ) : # перебираем все настройки биржи по парам

  symbol = eI[j]['symbol'] # название торговой пары
  
  for k in range( 6 ) :

    if eI[j]['filters'][k]['filterType'] == 'LOT_SIZE' :
 
      sett[symbol] = { 
      'id'              : j, 
      'tickSize'        : float( eI[j]['filters'][0]['tickSize'] ), # выписываем из настроек размер тика
      'lot_size'        : eI[j]['filters'][k]['minQty'] , # минимальный объем
      #'price_precision' : eI[j]['baseAssetPrecision'],  # количество знаков после запятой
      'price_precision' : int(round(-math.log(float( eI[j]['filters'][0]['tickSize']), 10), 0)),
      'lot_precision'   : int(round(-math.log(float(eI[j]['filters'][k]['stepSize']   ), 10), 0)),  # LOTS количество знаков после запятой 
      
      'quote_lot_precision': int(eI[j]['quotePrecision']),
      'status'          : eI[j]['status'], # разрешено ли торговать
      'baseAsset'       : eI[j]['baseAsset'], # базовая валюта
      'quoteAsset'      : eI[j]['quoteAsset'], # квотируемая валюта
      'eI'              : eI[j]
       } # словарь, где по названию торговой пары ищем её настройки
         
#print(sett)

In [ ]:
# ФИЛЬТРЫ : Поиск списка пар с хорошим спрэдом


pairs = {}

def find_pairs_by_spread( baseSymbol, quoteSymbol, min_spread_in_percent, min_spread_in_pips ) :
    global pairs
    pairs = {}
    tBT = bot1.tickerBookTicker() # Лучшие цены покупки/продажи

    for i in range( len(tBT) ): # перебираем все цены покупки
        symbol = tBT[i]['symbol']

        if sett[ symbol ]['quoteAsset'] == quoteSymbol or quoteSymbol == '' :
            if sett[ symbol ]['baseAsset'] == baseSymbol or baseSymbol == '' :
                if sett[ symbol ]['status'] == 'TRADING' :
                    if float(tBT[i]['askPrice']) > 0 : # если цена не 0
                        
                        spread_in_percent = round( calc_spread(tBT[i]), 2)
                        if spread_in_percent >= min_spread_in_percent : # если нас устраивает % спрэда

                            spread = float(tBT[i]['askPrice']) - float(tBT[i]['bidPrice']) #цена// вычисляем размер спрэда
                            spread = round(spread/sett[symbol]['tickSize']) # приводим спрэд в пункты

                            if spread >= min_spread_in_pips: #тики//  если размер больше
                                #print('id:',i, ', s%:', spread_in_percent, ', s:',spread, tBT[i])
                                pairs[symbol] =  {'s%' : spread_in_percent, '    s' : spread, '    s/%' : round(spread/spread_in_percent), '    id' : i } # заполняем словарь из номеров пар и их размеров тика


                                print( symbol, pairs[symbol] )
#pairs



In [ ]:
def calc_spread(bt) :
  spread_in_percent = 100*(float(bt['askPrice']) - float(bt['bidPrice']))/float(bt['askPrice']) #%// вычисляем спрэд в % от текущей цены
  return spread_in_percent




In [ ]:
def roundf(flt, dig) :
  if dig == 0 :
    return math.floor(flt)
  else :
    p = pow(10,dig)
    return float(round(math.floor(flt * p)/p, dig))

def rounds(flt, dig) :
  f = '{:.'+str(dig)+'f}'
  return f.format(float(flt))

In [2]:
def printx(string) :
    #global client_report
    #client_report = client_report + string + '\n'

    if isWriteToFile :  
        f.write(string+'\n')
  
    print(datetime.now().strftime("%d.%m.%Y %H:%M:%S"), string) 

def order_params(o) :
    try :
        ret = o['symbol'] + ' origQty:' + rounds( o['origQty'],     sett[ o['symbol'] ]['lot_precision']) + ' exQty:'   + rounds( o['executedQty'], sett[ o['symbol'] ]['lot_precision'])
        return ret 
    except :
        return 'o NULL'

## Балансы

In [ ]:
# проверка балансов аккаунтов
db1_old = 0; db2_old = 0 
def get_balances() :
    

  bal = { 'bot1':{}, 'bot2' : {} }

  acc = bot1.account()['balances']
  #print('bot1')
  for i in range( len(acc) ):
    if acc[i]['asset'] == USD :
      bal['bot1'][ USD ]       = float(acc[i]['free'])
      #print(acc[i])
    if acc[i]['asset'] == sett[base_quote]['baseAsset'] :
      bal['bot1']['baseAsset']  = float(acc[i]['free'])
      #print(acc[i])
    if acc[i]['asset'] == sett[base_quote]['quoteAsset'] :
      bal['bot1']['quoteAsset'] = float(acc[i]['free'])
      #print(acc[i])
    if acc[i]['asset'] == 'BNB' :
      bal['bot1']['BNB'] = round(float(acc[i]['free']), 3)
      #print(acc[i])  



  acc = bot2.account()['balances']
  #print('bot2')
  for i in range( len(acc) ):
    if acc[i]['asset'] == USD :
      bal['bot2'][ USD ]       = float(acc[i]['free'])
      #print(acc[i])
    if acc[i]['asset'] == sett[base_quote]['baseAsset'] :
      bal['bot2']['baseAsset']  = float(acc[i]['free'])
      #print(acc[i])
    if acc[i]['asset'] == sett[base_quote]['quoteAsset'] :
      bal['bot2']['quoteAsset'] = float(acc[i]['free'])
      #print(acc[i])
    if acc[i]['asset'] == 'BNB' :
      bal['bot2']['BNB'] = round(float(acc[i]['free']), 3)
      #print(acc[i])  


  BT  = bot2.tickerBookTicker( symbol= base_quote )
  bal['bot1']['quote_in_base'] = float(bal['bot1']['quoteAsset']) / float(BT['askPrice'])
  bal['bot1']['base_in_quote'] = float(bal['bot1']['baseAsset']) * float(BT['bidPrice'])
  bal['bot2']['quote_in_base'] = float(bal['bot2']['quoteAsset']) / float(BT['askPrice'])
  bal['bot2']['base_in_quote'] = float(bal['bot2']['baseAsset']) * float(BT['bidPrice'])

# Вычисление рабочих и минимальных сумм для текущей цены в базовой и квотируемой валюте к USD
  if isBU : # B/U
    BT  = bot1.tickerBookTicker( symbol= base_usd )
    bal['base_work']    = Work_usd / float(BT['askPrice'])  # Вычисляем сколько нужно покупать базовых монет из USD
    bal['base_min']     = usd_min  / float(BT['askPrice']) 
     
    bal['bot1']['base_in_usd'] = float(bal['bot1']['baseAsset']) * float(BT['bidPrice']) 
    bal['bot2']['base_in_usd'] = float(bal['bot2']['baseAsset']) * float(BT['bidPrice'])
    
  else :    # U/B
    BT  = bot1.tickerBookTicker( symbol= usd_base )
    bal['base_work']    = Work_usd * float(BT['bidPrice'])  # Вычисляем сколько нужно покупать базовых монет из USD
    bal['base_min']     = usd_min  * float(BT['bidPrice']) 
    bal['bot1']['base_in_usd'] = float(bal['bot1']['baseAsset']) / float(BT['askPrice'])
    bal['bot2']['base_in_usd'] = float(bal['bot2']['baseAsset']) / float(BT['askPrice'])

  if busd_busd :
    bal['quote_work'] = Work_usd # Вычисляем сколько нужно покупать квотируемой валюты из USD
    bal['quote_min']  = usd_min
    bal['bot1']['quote_in_usd'] = float(bal['bot1']['quoteAsset']) 
    bal['bot2']['quote_in_usd'] = float(bal['bot2']['quoteAsset']) 
    
  else :
    if isQU : # Q/U
      BT  = bot2.tickerBookTicker( symbol= quote_usd )
      bal['quote_work'] = Work_usd/ float(BT['askPrice'])  # Вычисляем сколько нужно покупать квотируемой валюты из USD
      bal['quote_min']  = usd_min / float(BT['askPrice']) 
      bal['bot1']['quote_in_usd'] = float(bal['bot1']['quoteAsset']) * float(BT['bidPrice'])
      bal['bot2']['quote_in_usd'] = float(bal['bot2']['quoteAsset']) * float(BT['bidPrice'])
      

    else :    # U/Q
      BT  = bot2.tickerBookTicker( symbol= usd_quote )
      bal['quote_work'] = Work_usd* float(BT['bidPrice'])  # Вычисляем сколько получим квотируемой валютынужно при продаже USD 
      bal['quote_min']  = usd_min * float(BT['bidPrice']) 
      bal['bot1']['quote_in_usd'] = float(bal['bot1']['quoteAsset']) / float(BT['askPrice'])
      bal['bot2']['quote_in_usd'] = float(bal['bot2']['quoteAsset']) / float(BT['askPrice'])

  bal['bot1']['ALL_USD'] = roundf( bal['bot1']['quote_in_usd'] +  bal['bot1']['base_in_usd'], 2)  
  bal['bot2']['ALL_USD'] = roundf( bal['bot2']['quote_in_usd'] +  bal['bot2']['base_in_usd'], 2)

    
    # разница баланса с предыдущего вызова
  global db1_old; global db2_old    
  db1 = rounds(bal['bot1']['ALL_USD'] - db1_old, 2)
  db2 = rounds(bal['bot2']['ALL_USD'] - db2_old, 2)
  db1_old = bal['bot1']['ALL_USD']      
  db2_old = bal['bot2']['ALL_USD'] 
                   
                   
  b1 = 'b1 b:'+ str(bal['bot1']['baseAsset']) + '(' + rounds(bal['bot1']['base_in_usd'], 0) + ')' 
  b1 = b1 + ' q:' + rounds( bal['bot1']['quoteAsset'], 2)  + '(' + rounds(bal['bot1']['quote_in_usd'], 0) + ')'
  b1 = b1 + '   bq:' + str(bal['bot1']['ALL_USD'] ) +'  B:' +str(bal['bot1']['BNB']) 
  #b1 = b1 + '   -:' + rounds(bal['bot1']['ALL_USD'] - b1_start, 1)  + '  К:' + rounds(-100 * (bal['bot2']['ALL_USD'] - b2_start)/(bal['bot1']['ALL_USD'] - b1_start), 1)
  b1 = b1 + '   <>:' + db1

  b2 = 'b2 b:'+ str(bal['bot2']['baseAsset']) + '(' + rounds(bal['bot2']['base_in_usd'], 0) + ')' 
  b2 = b2 + ' q:' + rounds( bal['bot2']['quoteAsset'], 2) + '(' + rounds(bal['bot2']['quote_in_usd'], 0) + ')' 
  b2 = b2 + '   bq:' + str(bal['bot2']['ALL_USD'] ) +'  B:' +str(bal['bot2']['BNB']) 
  #b2 = b2 + '   +:' + rounds(bal['bot2']['ALL_USD'] - b2_start, 1) + '  %:' + rounds(100 * (bal['bot2']['ALL_USD'] - b2_start)/b1_start, 1)
  b2 = b2 + '   <>:' + db2

  printx(b1)
  printx(b2)
  
                  

  global client_report
  client_report = client_report + b1 + '\n' + b2 + '\n\n' 
  
  return bal

In [ ]:
# Выяснить минимальный баланс базовой валюты для перекидки

# перед покупкой : найти минимальный баланс бота1 и бота2 для перекидки
def get_min_base_before_buy(BT) :
  bal = get_balances()
  if float(bal['bot1']['baseAsset'])  * float(BT['bidPrice']) < float(bal['bot2']['quoteAsset']) :
    return kb * float(bal['bot1']['baseAsset'])
  else :
    return kb * float(bal['bot2']['quoteAsset']) / float(BT['bidPrice'])

# перед продажей : найти минимальный баланс бота1 и бота2 для перекидки
def get_min_base_before_sell(BT) :
  bal = get_balances()
  if float(bal['bot2']['baseAsset'])  * float(BT['bidPrice']) < float(bal['bot1']['quoteAsset']) :
    return kb * float(bal['bot2']['baseAsset'])
  else :
    return kb * float(bal['bot1']['quoteAsset']) / float(BT['bidPrice'])



In [ ]:
def get_new_Work_usd() :
  global bal
  bal = get_balances()
  
  global Work_usd 
  Work_usd = 0.9 * min( [ bal['bot1']['quote_in_usd'] +  bal['bot1']['base_in_usd'] , bal['bot2']['quote_in_usd'] +  bal['bot2']['base_in_usd']  ] )
  print('New W_usd: ' + rounds(Work_usd, 2 ) )

## BUSD to BNB

In [ ]:

def busd_to_bnb() :
  bal = get_balances()

  BNB_min = 0.02
  qty = 20


  if bal['bot1']['BNB'] < BNB_min   and  bal['bot1']['quoteAsset'] > 100:
    print('b1 busd_to_bnb:', qty )
    try :
      o = bot1.createOrder(
        symbol        = 'BNBBUSD', 
        quoteOrderQty = rounds( qty * 0.9995, sett[ 'BNBBUSD' ]['quote_lot_precision'] ),  # quoteOrderQty - когда выставляем объем не по базовой валюте, а по квотируемой
        recvWindow    = 5000, side = 'BUY', type  = 'MARKET'
        )
      #print(o)
      print('b1 setOrder', order_params(o1))   
    except :
      print('b1 ERROR busd_to_bnb:', qty )  
  else :
    print('b1 BNB ok')

  if bal['bot2']['BNB'] < BNB_min :
    try :
      print('b2 busd_to_bnb:', qty )
      o = bot2.createOrder(
        symbol        = 'BNBBUSD', 
        quoteOrderQty = rounds( qty * 0.9995, sett[ 'BNBBUSD' ]['quote_lot_precision'] ),  # quoteOrderQty - когда выставляем объем не по базовой валюте, а по квотируемой
        recvWindow    = 5000, side = 'BUY', type  = 'MARKET'
        )
      #print(o)
      print('b2 setOrder', order_params(o1))   
    except :
      print('b2 ERROR busd_to_bnb:', qty )  
  else :
    print('b2 BNB ok')

 

  


## Ордеры

### Перекидки между Quote и USD

In [ ]:
# Перегоняем Quote в USD
def quote_to_usd(bot, qty) :
  
  if quote_usd == 'BUSDBUSD' : # or quote_usd == 'BNBBUSD':
    o={'не перекидываем', quote_usd}
    print( order_params(o) )
    return o

  if isQU :
    # Продаём в USD
    print(bot.bname, ' quote_to_usd:',quote_usd, qty )
    o = bot.createOrder(
      symbol      = quote_usd, 
      quantity    = rounds( qty * 0.999, sett[ quote_usd ]['lot_precision'] ),
      recvWindow  = 5000, side = 'SELL', type  = 'MARKET'
      )
    print( order_params(o) )
    return o

  else :    
    # Покупаем в USD
    print(bot.bname, ' quote_to_usd:',usd_quote, qty )
    o = bot.createOrder(
      symbol        = usd_quote, 
      quoteOrderQty = rounds( qty * 0.999, sett[ usd_quote ]['quote_lot_precision'] ),  # quoteOrderQty - когда выставляем объем не по базовой валюте, а по квотируемой
      recvWindow    = 5000, side = 'BUY', type  = 'MARKET'
      )
    print( order_params(o) )
    return o


# Перегоняем Base в USD
def base_to_usd(bot, qty) :
  
  if base_usd == 'BUSDBUSD' : # or base_usd == 'BNBBUSD':
    o={'не перекидываем', base_usd}
    print(o)
    return o

  if isBU :
    # Продаём Base в USD
    print(bot.bname, ' base_to_usd:',base_usd, qty )
    o = bot.createOrder(
      symbol      = base_usd, 
      quantity    = rounds( qty * 0.999, sett[ base_usd ]['lot_precision'] ),
      recvWindow  = 5000, side = 'SELL', type  = 'MARKET'
      )
    print( order_params(o) )
    return o

  else :    
    # Покупаем из Base в USD
    print(bot.bname, ' base_to_usd:',usd_base, qty )
    o = bot.createOrder(
      symbol        = usd_base, 
      quoteOrderQty = rounds( qty * 0.999, sett[ usd_base ]['quote_lot_precision'] ),  # quoteOrderQty - когда выставляем объем не по базовой валюте, а по квотируемой
      recvWindow    = 5000, side = 'BUY', type  = 'MARKET'
      )
    print( order_params(o) )
    return o
    
# Перегоняем USD в Base 
def usd_to_base(bot, qty) :


  if base_usd == 'BUSDBUSD':
    o={'не перекидываем ', base_usd }
    print( order_params(o) )
    return o
  
  if isBU :
    # Покупаем Base из USD
    print(bot.bname, ' usd_to_base:',base_usd, qty )
    o = bot.createOrder(
      symbol        = base_usd, 
      quoteOrderQty = rounds( qty * 0.999, sett[ base_usd ]['quote_lot_precision'] ),
      recvWindow    = 5000, side = 'BUY', type  = 'MARKET'
      )
    print( order_params(o) )
    return o
    
  else :
    
    # Продаём  из USD в Base
    print(bot.bname, ' usd_to_base:',usd_base, qty )
    o = bot.createOrder(
      symbol        = usd_base, 
      quantity      = rounds( qty * 0.999, sett[ usd_base ]['lot_precision'] ),  # quoteOrderQty - когда выставляем объем не по базовой валюте, а по квотируемой
      recvWindow    = 5000, side = 'SELL', type  = 'MARKET'
      )
    print( order_params(o) )
    return o

# Перегоняем USD в Quote
def usd_to_quote(bot, qty) :

  if quote_usd == 'BUSDBUSD' : #or quote_usd == 'BNBBUSD':
    o={'не перекидываем ', quote_usd}
    print( order_params(o) )
    return o

  if isQU :
    # Покупаем Quote из USD
    print(bot.bname, ' usd_to_quote:',quote_usd, qty )
    o = bot.createOrder(
      symbol        = quote_usd, 
      quoteOrderQty = rounds( qty * 0.999, sett[ quote_usd ]['quote_lot_precision'] ),
      recvWindow    = 5000, side = 'BUY', type  = 'MARKET'
      )
    print( order_params(o) )
    return o
    
  else :
    
    # Продаём  из USD в Quote
    print(bot.bname, ' usd_to_quote:',usd_quote, qty )
    o = bot.createOrder(
      symbol        = usd_quote, 
      quantity      = rounds( qty * 0.999, sett[ usd_quote ]['lot_precision'] ),  # quoteOrderQty - когда выставляем объем не по базовой валюте, а по квотируемой
      recvWindow    = 5000, side = 'SELL', type  = 'MARKET'
      )
    print( order_params(o) )
    return o








# Перегоняем quote в Base 
def quote_to_base(bot, qty) :  # qty is quote_to_convert
    print(bot.bname, ' quote_to_base:', base_quote, qty )
    o = bot.createOrder(
      symbol        = base_quote, 
      quoteOrderQty = rounds( qty * 0.999, sett[ base_quote ]['quote_lot_precision'] ),  # по квртируемой
      recvWindow    = 5000, side = 'BUY', type  = 'MARKET'
      )
    print( order_params(o) )
    return o
    
# Перегоняем Base в quote 
def base_to_quote(bot, qty) :  # qty is base_to_convert
    print(bot.bname, ' base_to_quote:', base_quote, qty )
    o = bot.createOrder(
      symbol        = base_quote, 
      quantity      = rounds( qty * 0.999, sett[ base_quote ]['lot_precision'] ), # по базовой
      recvWindow    = 5000, side = 'SELL', type  = 'MARKET'
      )
    print( order_params(o) )
    return o

### Уравнять

In [ ]:
# Перебалансировка после Покупки 
def after_buy() :

  # Запросить балансы bot1 и bot2 usd, 
  bal = get_balances()
  #Qmm = get_quote_min_max()


# Остаток бота1 перегоняем base в quote
  if bal['bot1']['quoteAsset'] + bal['quote_min'] < bal['quote_work'] :  # Если quote меньше рабочей на минимальный размер 
    print('b1 quote меньше рабочей суммы')
    
    base_to_convert = bal['base_work']  - bal['bot1']['quote_in_base']  # столько нехватает квотируемой валюты

    if bal['bot1']['baseAsset'] < base_to_convert : 
      base_to_convert = bal['bot1']['baseAsset']

    if base_to_convert >= bal['base_min'] :
      print('b1 переливаем base в quote', base_to_convert)
      base_to_quote(bot1, base_to_convert)

# Остаток бота2 перегоняем quote в base
  if bal['bot2']['baseAsset'] + bal['base_min'] < bal['base_work'] :  # Если base меньше рабочей на минимальный размер 
    print('b2 base меньше рабочей суммы')
    
    quote_to_convert  = bal['quote_work']  - bal['bot2']['base_in_quote'] 

    if bal['bot2']['quoteAsset'] < quote_to_convert :
      quote_to_convert = bal['bot2']['quoteAsset']

    if quote_to_convert >= bal['quote_min'] :
      print('b2 переливаем quote в base', quote_to_convert)
      quote_to_base(bot2, quote_to_convert)



# Перебалансировка после Продажи 
def after_sell() :
  
  # Запросить балансы bot1 и bot2 usd, 
  bal = get_balances()

# Остаток бота1 перегоняем quote в base
  if bal['bot1']['baseAsset'] + bal['base_min'] < bal['base_work'] :  # Если base меньше рабочей на минимальный размер 
    print('b1 base меньше рабочей суммы')
    #quote_add = bal['quote_work'] - bal['bot1']['quoteAsset'] # нужно докинуть 
    quote_to_convert  = bal['quote_work']  - bal['bot1']['base_in_quote'] 

    if bal['bot1']['quoteAsset'] < quote_to_convert :
      quote_to_convert = bal['bot1']['quoteAsset']

    if quote_to_convert >= bal['quote_min'] :
      print('b1 переливаем quote в base', quote_to_convert)
      quote_to_base(bot1, quote_to_convert)
        
# Остаток бота2 перегоняем base в quote.        
  if bal['bot2']['quoteAsset'] + bal['quote_min'] < bal['quote_work'] :  # Если quote меньше рабочей на минимальный размер 
    print('b2 quote меньше рабочей суммы')
    #quote_add = bal['quote_work'] - bal['bot2']['quoteAsset'] # нужно докинуть 
    base_to_convert  = bal['base_work']  - bal['bot2']['quote_in_base']

    if bal['bot2']['baseAsset'] < base_to_convert :
      base_to_convert = bal['bot2']['baseAsset']

    if base_to_convert >= bal['base_min'] :
      print('b2 переливаем base в quote', base_to_convert)
      base_to_quote(bot2, base_to_convert)





In [ ]:
# Перебалансировка после Покупки 
def before_sell() :

  # Запросить балансы bot1 и bot2 usd, 
  bal = get_balances()
  #Qmm = get_quote_min_max()


# Остаток бота1 перегоняем base в quote
  if bal['bot1']['quoteAsset'] + bal['quote_min'] < bal['quote_work'] :  # Если quote меньше рабочей на минимальный размер 
    print('b1 quote меньше рабочей суммы')
    
    base_to_convert = bal['base_work']  - bal['bot1']['quote_in_base']  # столько нехватает квотируемой валюты

    if bal['bot1']['baseAsset'] < base_to_convert : 
      base_to_convert = bal['bot1']['baseAsset']

    if base_to_convert >= bal['base_min'] :
      print('b1 переливаем base в quote', base_to_convert)
      base_to_usd(bot1, base_to_convert)

# Остаток бота2 перегоняем quote в base
  if bal['bot2']['baseAsset'] + bal['base_min'] < bal['base_work'] :  # Если base меньше рабочей на минимальный размер 
    print('b2 base меньше рабочей суммы')
    
    quote_to_convert  = bal['quote_work']  - bal['bot2']['base_in_quote'] 

    if bal['bot2']['quoteAsset'] < quote_to_convert :
      quote_to_convert = bal['bot2']['quoteAsset']

    if quote_to_convert >= bal['quote_min'] :
      print('b2 переливаем quote в base', quote_to_convert)
      usd_to_base(bot2, quote_to_convert)



# Перебалансировка после Продажи 
def before_buy() :
  
  # Запросить балансы bot1 и bot2 usd, 
  bal = get_balances()

# бот1 перегоняем quote в base. если base меньше рабочей суммы
  if bal['bot1']['baseAsset'] + bal['base_min'] < bal['base_work'] :  # Если base меньше рабочей на минимальный размер 
    print('b1 base меньше рабочей суммы')
    #quote_add = bal['quote_work'] - bal['bot1']['quoteAsset'] # нужно докинуть 
    quote_to_convert  = bal['quote_work']  - bal['bot1']['base_in_quote'] 

    if bal['bot1']['quoteAsset'] < quote_to_convert :
      quote_to_convert = bal['bot1']['quoteAsset']

    if quote_to_convert >= bal['quote_min'] :
      print('b1 переливаем quote в base', quote_to_convert)
      usd_to_base(bot1, quote_to_convert)
        
# Остаток бота2 перегоняем base в quote.        
  if bal['bot2']['quoteAsset'] + bal['quote_min'] < bal['quote_work'] :  # Если quote меньше рабочей на минимальный размер 
    print('b2 quote меньше рабочей суммы')
    #quote_add = bal['quote_work'] - bal['bot2']['quoteAsset'] # нужно докинуть 
    base_to_convert  = bal['base_work']  - bal['bot2']['quote_in_base']

    if bal['bot2']['baseAsset'] < base_to_convert :
      base_to_convert = bal['bot2']['baseAsset']

    if base_to_convert >= bal['base_min'] :
      print('b2 переливаем base в quote', base_to_convert)
      base_to_usd(bot2, base_to_convert)





### Ждем спрэд

In [ ]:
last_spread = 0
ls1 = 0
ls2 = 0

def wait_spread( ):
    # Вычислить спрэд на выбранной паре
    # Если спрэд не большой
    # Пауза и повтор проверки
    # Если спрэд устраивает по размеру  
    # возвращаем   цены покупки/продажи
    global last_spread
    global ls1, ls2

    while 1 : # бесконечно повторяем
        #%run Break.ipynb
        #if BREAK : BREAK = False; break
        
        
        start_time = time.time()
        try :
            BT = bot2.tickerBookTicker( symbol = base_quote ) # Лучшие цены покупки/продажи
        except Exception as e:
            print(e)
            time.sleep( time_wait_spread ) # иначе ждем и снова повторяем
            continue

        ping = (time.time() - start_time)

        spread_in_percent = round( calc_spread(BT), 2)
        #print(spread_in_percent)
        
        ls2 = ls1
        ls1 = spread_in_percent
            
        if spread_in_percent != last_spread :
            
            
            last_spread = spread_in_percent
            print(base_quote, '. спрэд:', spread_in_percent , '  ping:', rounds(ping * 1000 , 0), '  bid:', float(BT['bidPrice']), '  ask:', float(BT['askPrice'])  ) 

        if ls2 >= min_spread_in_percent * 0.49 and ls1 >= min_spread_in_percent * 0.73 :   
            if spread_in_percent >= min_spread_in_percent : # + random.random() * 0.1 : # если спрэд больше минимального
                last_spread = 0
                return BT # выходим из цикла

        time.sleep( time_wait_spread ) # иначе ждем и снова повторяем

### перекидка на покупке

In [1]:

new_price_bs = 0
def buy_sell_21( tBT, in_qty ) :
  
  qty = in_qty
  #qty   = get_min_base_before_buy(tBT) * 0.9995 
  qty   = roundf( qty , sett[ base_quote ]['lot_precision'] )
  qty_s = rounds( qty , sett[ base_quote ]['lot_precision'] )


  if qty < bal['base_min'] :
    print('qty < usd_min/// -->')
    return True

  #if qty < usd_min :

  #tBT       = bot1.tickerBookTicker( symbol = smb ) # Лучшие цены покупки/продажи для символа
  
  global new_price_bs
  new_price_bs = 0  #!!!!!!!!!!!!!!!!!!!!
  if new_price_bs > 0 :
    price_dwn = new_price_bs
    new_price_bs = -1
  else :
    price_dwn = float(tBT['bidPrice']) + 1.01 * float(sett[base_quote]['tickSize']) # цена на 1 пункт выше лучшей цены покупки
    price_dwn   = roundf(price_dwn, int(sett[base_quote]['price_precision']))  # Округляем до заданной в паре точности
    price_dwn_s = rounds(price_dwn, int(sett[base_quote]['price_precision']))  # Округляем до заданной в паре точности

  print('b2 BUY ',base_quote, 'bid:', price_dwn_s, 'qty:', qty_s)

  try :
    o2 = bot2.createOrder(
      symbol      = base_quote,
      recvWindow  = 5000,
      side        = 'BUY',
      type        = 'LIMIT',
      timeInForce = 'GTC', # (Good Till Cancelled) – ордер будет висеть до тех пор, пока его не отменят.
      quantity    = qty_s,
      price       = price_dwn_s  # Округляем до заданной в паре точности
    )

    print('b2 setOrder', order_params(o2))   
  except Exception as e:
    print(e)
    return False

  #time.sleep(0.2)  
  
  time.sleep(time_b2_b1 * random.random())

  tBT = bot2.tickerBookTicker( symbol= base_quote ) # Лучшие цены покупки/продажи для символа

  if float(tBT['bidPrice']) > price_dwn and float(tBT['bidQty']) <= qty * eat_vol :
    print('сидит объем:', tBT['bidPrice'], '  Vol:', tBT['bidQty'])

  if float(tBT['bidPrice']) > price_dwn and float(tBT['bidQty']) > qty * eat_vol :
  #if False :
    # кто-то поставил выше, надо отменить ордер и подождать
    print('b2 перебил bid:', rounds(tBT['bidPrice'] , int(sett[base_quote]['price_precision'])),
          '  qty:', rounds(tBT['bidQty'] , sett[ base_quote ]['lot_precision'] ))
    
    time.sleep(5 * random.random() + 0.1)

    try :
      o2 = bot2.cancelOrder(
      orderId = o2['orderId'],
      symbol  = base_quote ,
      )
    
      print('b2 cancelOrder', order_params(o2))   
    except :
      print('b2 o2 нечего было закрывать')
      return True

    #if new_price_bs == 0 :  
    #  new_price_bs = float(tBT['bidPrice'])  + float(sett[base_quote]['tickSize'])
    #  new_price_bs = round(new_price_bs, int(sett[base_quote]['price_precision']))
    #  print('новая цена', new_price_bs)
    #if new_price_bs == -1 :
    #  new_price_bs = 0  

    return False

  #elif float(tBT['bidQty']) != volume : 
  #  print('воткнули на нашу цену')

  else :
    print('b1 try SELL ', qty_s)

    try : # патаемся откупить ботом1 ордер бота2
      o1 = bot1.createOrder(
        symbol      = base_quote,
        recvWindow  = 5000,
        side        = 'SELL',
        type        = 'LIMIT',
        timeInForce = 'GTC', #'FOK', # (Fill-Or-Kill) Либо будет куплено все указанное количество немедленно, либо не будет куплено вообще ничего, ордер отменится.
        
        quantity    = qty_s,
        price       = price_dwn_s  # Округляем до заданной в паре точности
      )
    except : # если не получилось откупить, то
      print('b1 не откупил')
      try :  # пытаемся отменить ордер бота2
        print('b2 отмена ордера. id:', o2['orderId'])
        bot2.cancelOrder(
          orderId = o2['orderId'],
          symbol  = base_quote ,
        )
      except Exception as e:
        print(e) 
      return False  # возвращаемся на ожидание спрэда
  
  if o1.get('status', 'FILLED') == 'FILLED' :
    print('b1 Откуп SELL = FILLED ')
    return True
  else :
    try :
        bot2.cancelOrder(
          orderId = o2['orderId'],
          symbol  = base_quote ,
        )
        print('b1 SELL not FILLED')
        print('b1 cancelOrder', order_params(o2))   
    except Exception as e:
        print(e)
    return False

In [ ]:
async def _buy_sell_21( tBT, in_qty ) :
  
  async def _o2():
    try :
      o2 = bot2.createOrder(
        symbol      = base_quote,
        recvWindow  = 5000,
        side        = 'BUY',
        type        = 'LIMIT',
        timeInForce = 'GTC', # (Good Till Cancelled) – ордер будет висеть до тех пор, пока его не отменят.
        quantity    = qty_s,
        #quantity    = round( Qmm['base_work'] * 0.9995, sett[ base_quote ]['lot_precision'] ),
        price       = price_dwn_s
        )
      
      return o2
    except Exception as e:
      print('b2 ', e)
      return False  

  async def _o1():
    # патаемся откупить ботом1 ордер бота2
    print('bot1 try SELL ', qty)
    try :
      o1 = bot1.createOrder(
        symbol      = base_quote,
        recvWindow  = 5000,
        side        = 'SELL',
        type        = 'LIMIT',
        timeInForce = 'GTC', #'FOK', # (Fill-Or-Kill) Либо будет куплено все указанное количество немедленно, либо не будет куплено вообще ничего, ордер отменится.
        
        quantity    = qty_s,
        #quantity    = round( Qmm['base_work'] * 0.9995, sett[ base_quote ]['lot_precision'] ),
        price       = price_dwn_s
      )
      
      
      return o1
    except Exception as e:
      print('b1 ', e)
      return False

  qty = in_qty
  #qty   = get_min_base_before_buy(tBT) * 0.9995 
  qty   = roundf( qty , sett[ base_quote ]['lot_precision'] )
  qty_s = rounds( qty , sett[ base_quote ]['lot_precision'] )

  if qty < bal['base_min'] :
    print('qty < usd_min/// -->')
    return True

  #tBT       = bot1.tickerBookTicker( symbol = smb ) # Лучшие цены покупки/продажи для символа
  
  price_dwn = float(tBT['bidPrice']) + 1.01 * float(sett[base_quote]['tickSize']) # цена на 1 пункт выше лучшей цены покупки
  price_dwn   = roundf(price_dwn, int(sett[base_quote]['price_precision']))  # Округляем до заданной в паре точности
  price_dwn_s = rounds(price_dwn, int(sett[base_quote]['price_precision']))  # Округляем до заданной в паре точности

  print('b2 BUY ', base_quote, 'price_dwn:', price_dwn, 'quantity:', qty)


  o1= {}
  o2= {}
  
  task_o2 = asyncio.create_task(
    _o2()
    )

    #time.sleep(0.1)

  task_o1 = asyncio.create_task(
    _o1()
    )

  o2 = await task_o2
  o1 = await task_o1

  print('b2 setOrder', order_params(o2))    
  print('b1 setOrder', order_params(o1))    

  if o2 == False :
    print('o2 False') 
    return False

  if o1 == False :
  #except : # если не получилось откупить, то
    print('bot1 не откупил')
    try :  # пытаемся отменить ордер бота2
      print('bot2 отмена ордера. id:', o2['orderId'])
      bot2.cancelOrder(
        orderId = o2['orderId'],
        symbol  = base_quote ,
      )
      return False  # возвращаемся на ожидание спрэда
    except :        # если не получилось оменить, то
      return False  # возвращаемся на ожидание спрэда
  
  if o1.get('status', 'FILLED') == 'FILLED' :
    print('b1 Откуп SELL = FILLED ')
    return True
  else :
    try :
        bot2.cancelOrder(
          orderId = o2['orderId'],
          symbol  = base_quote ,
        )
        print('b1 SELL not FILLED')
        print('b1 cancelOrder', order_params(o2))   
    except Exception as e:
        print(e)
    return False

### перекидка на продаже

In [ ]:

new_price = 0
def sell_buy_21(in_qty) :
  global new_price 

  tBT = bot2.tickerBookTicker( symbol= base_quote ) # Лучшие цены покупки/продажи для символа

  qty = in_qty #get_min_base_before_sell(tBT) * 0.9995
  qty   = roundf(qty , sett[ base_quote ]['lot_precision'] )
  qty_s = rounds(qty , sett[ base_quote ]['lot_precision'] )

  if qty < bal['base_min'] :
    print('qty < usd_min  -->')
    return True  
    
  #tBT       = bot1.tickerBookTicker( symbol= base_quote ) # Лучшие цены покупки/продажи для символа
  if new_price > 0 :
    price_up = new_price
    new_price = -1
  else :
    price_up  = float(tBT['askPrice']) - 1.01 * float(sett[base_quote]['tickSize'])   # цена на 1 пункт ниже лучшей цены продажи
    price_up    = roundf(price_up, int(sett[base_quote]['price_precision']))
    price_up_s  = rounds(price_up, int(sett[base_quote]['price_precision']))
  

  print('b2 SELL ', base_quote, 'ask:', price_up_s, 'qty:', qty_s)

  try : 
    o2 = bot2.createOrder(
    symbol      = base_quote,
    recvWindow  = 5000,
    side        = 'SELL',
    type        = 'LIMIT',
    timeInForce = 'GTC', # (Good Till Cancelled) – ордер будет висеть до тех пор, пока его не отменят.
    quantity    = qty_s,
    price       = price_up_s
    )

    print('b2 setOrder', order_params(o2)) 
  except :
    return False
  
  time.sleep(time_b2_b1 * random.random())

  tBT = bot2.tickerBookTicker( symbol= base_quote ) # Лучшие цены покупки/продажи для символа
  
  
  if float(tBT['askPrice']) < price_up and float(tBT['askQty']) <= qty * eat_vol :
    print('сидит объем:', tBT['askPrice'], '  Vol:', tBT['askQty'])

  if float(tBT['askPrice']) < price_up and float(tBT['askQty']) > qty * eat_vol :
  #if False :
    # кто-то поставил ниже, надо отменить ордер и подождать
    print('b2 перебил ask:', rounds(tBT['askPrice'] , int(sett[base_quote]['price_precision'])),
          '  qty:', rounds(tBT['askQty'] , sett[ base_quote ]['lot_precision'] ))
    
    #time.sleep(5 * random.random() + 0.1)
    
    try:
      o2 = bot2.cancelOrder(
      orderId = o2['orderId'],
      symbol  = base_quote ,
      )
    
      print('b2 cancelOrder', order_params(o2))    
    except :
      print('b2 o2 нечего было закрывать')
      return True
    
    #if new_price == 0 :  
      #new_price = float(tBT['askPrice'])  - float(sett[base_quote]['tickSize'])
      #new_price = round(new_price, int(sett[base_quote]['price_precision']))
      #print('новая цена', new_price)
    #if new_price == -1 :
    #  new_price = 0  

    return False
  #elif float(tBT['askQty']) > qty :
    #print('объем больше нешего ', tBT['askQty'] )
    #print('bot2 cancelOrder', 
    #bot2.cancelOrder(
    #orderId = o2['orderId'],
    #symbol  = base_quote ,
    #))
    #return False
  else :
    print('b1 try BUY ', qty_s)
    new_price = 0

    try :
      o1 = bot1.createOrder(
        symbol      = base_quote,
        recvWindow  = 5000,
        side        = 'BUY',
        type        = 'LIMIT',
        timeInForce = 'GTC', #'FOK', # (Fill-Or-Kill) Либо будет куплено все указанное количество немедленно, либо не будет куплено вообще ничего, ордер отменится.
        quantity    = qty_s,
        price       = price_up_s
      )
    except :
      try :
        bot2.cancelOrder(
          orderId = o2['orderId'],
          symbol  = base_quote ,
        )  
      except Exception as e:
        print(e)
      return False  

  if o1.get('status', 'FILLED') == 'FILLED' :
    print('b1 Откуп BUY = FILLED ')
    return True
  else :
    try :
        bot2.cancelOrder(
          orderId = o2['orderId'],
          symbol  = base_quote ,
        )
        print('b1 BUY not FILLED')
    except Exception as e:
        print(e)
    return False

In [ ]:

new_price = 0
async def _sell_buy_21(in_qty) :
  global new_price 

  tBT = bot2.tickerBookTicker( symbol= base_quote ) # Лучшие цены покупки/продажи для символа
  
  qty = in_qty #get_min_base_before_sell(tBT) * 0.9995
  qty   = roundf(qty , sett[ base_quote ]['lot_precision'] )
  qty_s = rounds(qty , sett[ base_quote ]['lot_precision'] )
    
  if qty < bal['base_min'] :
    print('qty < usd_min  -->')
    return True
  
  #tBT       = bot1.tickerBookTicker( symbol= base_quote ) # Лучшие цены покупки/продажи для символа
  if new_price > 0 :
    price_up = new_price
    new_price = -1
  else :
    price_up  = float(tBT['askPrice']) - 1.01 * float(sett[base_quote]['tickSize'])   # цена на 1 пункт ниже лучшей цены продажи
    price_up    = roundf(price_up, int(sett[base_quote]['price_precision']))
    price_up_s  = rounds(price_up, int(sett[base_quote]['price_precision']))
  
  

  print('b2 SELL ',base_quote, 'price_up:', price_up, 'quantity:', qty)

  async def _o2():
    try :
      o2 = bot2.createOrder(
        symbol      = base_quote,
        recvWindow  = 5000,
        side        = 'SELL',
        type        = 'LIMIT',
        timeInForce = 'GTC', # (Good Till Cancelled) – ордер будет висеть до тех пор, пока его не отменят.
        quantity    = qty_s,
        price       = price_up_s
        )
      
      return o2
    except Exception as e:
      print('b2 ', e)
      return False

  async def _o1():
    try :
      o1 = bot1.createOrder(
        symbol      = base_quote,
        recvWindow  = 5000,
        side        = 'BUY',
        type        = 'LIMIT',
        timeInForce = 'GTC', #'FOK', # (Fill-Or-Kill) Либо будет куплено все указанное количество немедленно, либо не будет куплено вообще ничего, ордер отменится.
        quantity    = qty_s,
        price       = price_up_s
        )
      
      return o1
    except Exception as e:
      print('b1 ', e)
      return False

  o1= {}
  o2= {}

  
  task_o2 = asyncio.create_task(
    _o2()
    )

    #time.sleep(0.1)

  task_o1 = asyncio.create_task(
    _o1()
    )

    
  o2 = await task_o2
  o1 = await task_o1

  print('b2 setOrder', order_params(o2))    
  print('b1 setOrder', order_params(o1))    

  if o2 == False :
    print('o2 False')
    return False
    
  if o1 == False :
  #except :
    print('bot1 не откупил')
    try :
      print('bot2 отмена ордера. id:', o2['orderId'])
      bot2.cancelOrder(
        orderId = o2['orderId'],
        symbol  = base_quote ,
      )
      return False  
    except :
      return False  

  if o1.get('status', 'FILLED') == 'FILLED' :
    print('b1 Откуп BUY = FILLED ')
    return True
  else :
    try :
        bot2.cancelOrder(
          orderId = o2['orderId'],
          symbol  = base_quote ,
        )
        print('b1 BUY not FILLED')
    except Exception as e:
        print(e)
    return False

### Закрытие ордеров О1 и О2

In [ ]:
# Отмена ордеров в любом случае
def cancel_orders(bot) :
    oo = bot.openOrders()

    for i in range( len(oo) ):
        try :
            bot.cancelOrder(
              orderId = oo[i]['orderId'],
              symbol  = oo[i]['symbol']
            )   
            print(f'{bot.bname} oo{i} закрыл остатки', order_params(oo[i]))  
        except Exception as e:
            print(e)
        

    

## Основной цикл

In [ ]:
async def _try_buy_sell() :
  while  1 :
    tBT  = bot1.tickerBookTicker( symbol= base_quote )
    in_qty = get_min_base_before_buy(tBT) * 0.999 
    
    BT = wait_spread() 
    
    if await _buy_sell_21( BT, in_qty ) == True :
      print('buy_sell() True \n')

      break
    else :
      print('buy_sell() False \n')

      cancel_orders(bot2)

      time.sleep( random.random() *10 ) # иначе ждем и снова повторяем
      continue
    
  time.sleep( 1 )
  cancel_orders(bot2)

  after_buy()


async def _try_sell_buy() :
  while 1 :  
    BT  = bot1.tickerBookTicker( symbol= base_quote )
    in_qty = get_min_base_before_sell(BT) * 0.999
    #BT  = bot2.tickerBookTicker( symbol= base_quote )
    
    if await _sell_buy_21( in_qty ) == True :  
      print('sell_buy() True \n')
      break
    else :
      print('sell_buy() False \n')

      cancel_orders(bot2)
    
      time.sleep(random.random() *10 + 5)
      continue
    
  cancel_orders(bot2)

  after_sell()
  time.sleep( 1 )
  

In [ ]:
def try_buy_sell() :  
  #bal = get_balances(base_quote)

  while  1 :
    tBT  = bot1.tickerBookTicker( symbol= base_quote )
    in_qty = get_min_base_before_buy(tBT) * 0.9995 
    
    BT = wait_spread() 
    
    if buy_sell_21( BT, in_qty ) == True :
      print('buy_sell() True \n')

      break
    else :
      print('buy_sell() False \n')

      cancel_orders(bot2)
      time.sleep( random.random() *10 ) # иначе ждем и снова повторяем
      continue
    
  time.sleep( 1 )
  cancel_orders(bot1)
  cancel_orders(bot2)

  after_buy()

def try_sell_buy() :
  while 1 :  
    BT  = bot1.tickerBookTicker( symbol= base_quote )
    in_qty = get_min_base_before_sell(BT) * 0.9995
    if sell_buy_21( in_qty ) == True :
      print('sell_buy() True \n')
      break
    else :
      print('sell_buy() False \n')

      cancel_orders(bot2)
    

      time.sleep(random.random() *10 + 5)
      continue
    
  cancel_orders(bot1)
  cancel_orders(bot2)

  after_sell()
  time.sleep( 1 )
 

## Три монеты

In [ ]:
# Три монеты

USD = 'BUSD'

def udToken(Token) :
  try : 
    sett[ Token + 'BUSD' ];
    return True
  except : 
    return False
    

def find_pairs(b, q, u) :
  global base_quote, base_usd, usd_base, quote_usd, usd_quote
  global isBU, isQU

  try : 
    base_quote = b+q
    sett[ b+q ]; 
    print('base_quote:', base_quote)
    
  except : 
    quote_base = q+b
    print('quote_base:', quote_base)

  try : 
    base_usd = b+u
    sett[ b+u ]; 
    print('base_usd:', base_usd, 'True'); 
    isBU = True; # True если base/usd    False, если Usd/base
    
  except : 
    usd_base = u+b
    print('usd_base:', usd_base, 'False'); 
    isBU = False; 

  try : 
    quote_usd = q+u
    sett[ q+u ]; 
    print('quote_usd:', quote_usd, 'True'); 
    isQU = True; # True если quote/usd    False, если usd/quote
    
  except : 
    isQU = False; 
    usd_quote = u+q
    print('usd_quote:', usd_quote, 'False'); 



In [ ]:
def getBalancesInUSD(bot) :
    
    tBT = bot2.tickerBookTicker( )
    acc = bot.account()['balances']
    
    res = []
    
    for i in range( len(acc) ):
        if float(acc[i]['free']) > 0 :
            
            if udToken(acc[i]['asset']) :
                ud = acc[i]['asset'] + 'BUSD'
                isUD = True
            else : 
                ud = 'BUSD' + acc[i]['asset']
                isUD = False
            
            res_i = {}
            if ud == 'BUSDBUSD' : res_i = { 'asset' : acc[i]['asset'] , 'free_usd' : round( float(acc[i]['free']) ,2)  , 'locked_usd' : round( float(acc[i]['locked']) ,2)  }

            for j in range( len(tBT) ) :
                if tBT[j]['symbol'] == ud :
                    if isUD : res_i = { 'asset' : acc[i]['asset'] , 'free_usd' : round( float(acc[i]['free']) * float(tBT[j]['bidPrice']) ,2)  , 'locked_usd' : round( float(acc[i]['locked']) * float(tBT[j]['bidPrice']) ,2) , 'free' : round( float(acc[i]['free'])), 'locked' : round( float(acc[i]['locked']))  }
                    else    : res_i = { 'asset' : acc[i]['asset'] , 'free_usd' : round( float(acc[i]['free']) / float(tBT[j]['askPrice']) ,2)  , 'locked_usd' : round( float(acc[i]['locked']) / float(tBT[j]['askPrice']) ,2) , 'free' : round( float(acc[i]['free'])), 'locked' : round( float(acc[i]['locked']))  }
            res.append(res_i)
            print(res_i)
    return res

In [1]:
def report(b1_start, b2_start, b2_) :
    
    a1 = 0
    a2 = 0

    print('bot1')
    acc1 = getBalancesInUSD(bot1)
    print('bot2')
    acc2 = getBalancesInUSD(bot2)

    for i in range( len(acc1) ) : 
            f = acc1[i].get( 'free_usd' , 0 ) + acc1[i].get( 'locked_usd' , 0 )
            a1 = a1 + f

    for i in range( len(acc2) ) : 
            f = acc2[i].get( 'free_usd' , 0 ) + acc2[i].get( 'locked_usd' , 0 )
            a2 = a2 + f

    print('')
    

    db1 = round( b1_start - a1 ,2)
    db2 = round( a2 - b2_start ,2)

    print(b1_API_KEY)
    print('было: ', b1_start)
    print(f'ушло: {db1} $')
    print('сейчас:', round( a1,2 ) )

    print('к нам пришло:', db2 )
    print(f' {db2}/{db1} = {  round( 100 * db2/db1, 2 ) } % \n')



    give = round( db2 * percent / 100 , 2)

    print(f'отдаем {percent} % : {give} $')
    print(f'клиент получил {round(100*give/b1_start)} % от депозита')

    print('отдавали: ', b2_, '. Остались должны: ', round(give - b2_) )

    print('\n')
    print(f'нам {100 - percent} % : {round(db2 - give, 1)} $')
    print('на нашем счете:', round(a2, 2) )

In [ ]:
# конвертируем из токена a в токен b

def atob(bot, a, b, qty, qty_in_A_or_B) :
    try : 
        pair = a+b
        sett[ pair ]; 
        print('ab:', pair)
        
        if qty_in_A_or_B == 1 or qty_in_A_or_B == 'a':
            print(bot.createOrder(
            symbol        = pair, 
            quantity      = rounds( qty * 0.9995, sett[ pair ]['lot_precision'] ),
            recvWindow    = 5000, side = 'SELL', type  = 'MARKET' ))
        else :
            print(bot.createOrder(
            symbol        = pair, 
            quoteOrderQty = rounds( qty * 0.9995, sett[ pair ]['lot_precision'] ),
            recvWindow    = 5000, side = 'SELL', type  = 'MARKET' ))

    except : 
        pair = b+a
        sett[ pair ];
        print('ba:', pair)
        
        if qty_in_A_or_B == 1 or qty_in_A_or_B == 'a' :
            print(bot.createOrder(
            symbol        = pair, 
            quoteOrderQty = rounds( qty * 0.9995, sett[ pair ]['lot_precision'] ),
            recvWindow    = 5000, side = 'BUY', type  = 'MARKET' ))
        else :
            print(bot.createOrder(
            symbol        = pair, 
            quantity      = rounds( qty * 0.9995, sett[ pair ]['lot_precision'] ),
            recvWindow    = 5000, side = 'BUY', type  = 'MARKET' ))
